Import all the required packages

In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D,Flatten,MaxPooling2D,Dense,Dropout

Check if you have tensorflow version more than 2.0

In [2]:
print(tf.__version__)

2.0.0


Download the dataset for the code. Assign the training and testing data directory in the variables train_dir and test_dir respectively

In [3]:
train_dir = 'maskdata/train/'
test_dir = 'maskdata/test/'

We need to initialize ImageDataGenerators so as to rescale the images. The original images consist of RGB pixel value in the range 0-255. These values are too high for the model to process. Thus by dividing the pixel values by 255 brings each value in the range 0-1 which is smaller and easier to work upon.

In [4]:
train = ImageDataGenerator(rescale=1/255.0)
test = ImageDataGenerator(rescale=1/255.0)

To obtain the images from the dataset and to classify each image according to the folder in which they are stored, we use the 'flow_from_directory' method.  

In [5]:
train_dataset = train.flow_from_directory(train_dir, target_size = (200,200),  class_mode ='binary')
test_dataset = test.flow_from_directory(test_dir, target_size = (200,200),  class_mode ='binary')

Found 1207 images belonging to 2 classes.
Found 334 images belonging to 2 classes.


Now that we have our data and classified them, we design out model for the prediction. We use the activation function 'sigmoid' since we have to make classification into two classes : namely withmask and withoutmask.

In [6]:
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(200 , 200, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [7]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

It is a good practice to use callbacks in our code. A callback is used during model fitting. When we reach an epoch with the mentioned value in the callback function, the model training stops even if some epochs are left.

In [8]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        print(logs.get('accuracy'))
        if(logs.get('accuracy')>0.96):
            print("\nReached 96% accuracy so cancelling training!")
            self.model.stop_training = True
        
callbacks = myCallback()

In [9]:
history = model.fit(train_dataset, 
                    epochs=20, 
                    validation_data=test_dataset ,
                    callbacks=[callbacks])

Train for 38 steps, validate for 11 steps
Epoch 1/20
38/38 [==============================] - 178s 5s/step - loss: 0.7199 - accuracy: 0.7945 - val_loss: 0.1502 - val_accuracy: 0.9461
Epoch 2/20
38/38 [==============================] - 175s 5s/step - loss: 0.1109 - accuracy: 0.9569 - val_loss: 0.1116 - val_accuracy: 0.9581
Epoch 3/20
37/38 [============================>.] - ETA: 4s - loss: 0.0929 - accuracy: 0.96940.968517

Reached 96% accuracy so cancelling training!
38/38 [==============================] - 173s 5s/step - loss: 0.0941 - accuracy: 0.9685 - val_loss: 0.1099 - val_accuracy: 0.9551


Now that we have trained our model and have a permissible accuracy of our model, we can predict on on images ,videos or use our webcam for prediction on live video streaming. But we need to preprocess them before predicting.We rescale and reshape the images. Here we are using a video and predicting on it. 

In [10]:
def prepImg(data):
    return cv2.resize(data,(200,200)).reshape(200,200,3)/255.0
 
classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [11]:
cap =cv2.VideoCapture("fast.mp4")
if (cap.isOpened()== False):  
    print("Error opening video  file") 
    
while(cap.isOpened()): 
    ret, frame = cap.read() 
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = classifier.detectMultiScale(gray)
    for (x,y,w,h) in faces :
        sliced = frame[y:y+h,x:x+w]
        sliced = prepImg(sliced)
        sliced = np.expand_dims(sliced , axis = 0)
        pred = model.predict(sliced)
        print(pred)
        if pred <0.5:
            cv2.rectangle(frame , (x,y) , (x+w,y+h) , (0,0,255) , 3)
            cv2.putText(frame, 'NO MASK', (x,y), cv2.FONT_HERSHEY_SIMPLEX ,  1, (0,0,255) , 2, cv2.LINE_AA) 
        else :
            cv2.rectangle(frame , (x,y) , (x+w,y+h) , (0,255,0) , 3)    
            cv2.putText(frame, 'MASK', (x,y), cv2.FONT_HERSHEY_SIMPLEX ,  1, (0,255,0) , 2, cv2.LINE_AA) 

    cv2.imshow('Frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
cv2.destroyAllWindows()

[[0.9878996]]
[[0.9889389]]
[[0.9889389]]
[[0.9878198]]
[[0.9889971]]
[[0.9892363]]
[[0.9875935]]
[[0.9843419]]
[[0.9866368]]
[[0.9845307]]
[[0.9855606]]
[[0.9882443]]
[[0.9834661]]
[[0.98415035]]
[[0.9920729]]
[[0.99086785]]
[[0.98941433]]
[[0.98422235]]
[[0.9798078]]
[[0.98331636]]
[[0.9831313]]
[[0.98496675]]
[[0.9843774]]
[[0.48884788]]
[[0.9808498]]
[[0.9809724]]
[[0.98407066]]
[[0.9772027]]
[[0.9818107]]
[[0.9843241]]
[[0.99085355]]
[[0.990836]]
[[0.9913667]]
[[0.98919606]]
[[0.9870962]]
[[0.9856952]]
[[0.98475236]]
[[0.9845889]]
[[0.97137535]]
[[0.9631478]]
[[0.00854731]]
[[0.7416767]]
[[0.52907777]]
[[0.16700062]]
[[0.0239353]]
[[0.01590835]]
[[0.00619407]]
[[0.0038557]]
[[0.00294279]]
[[0.00206496]]
[[0.00290489]]
[[0.01025569]]
[[0.01682397]]
[[0.01054952]]
[[0.01967985]]
[[0.02188615]]
[[0.01136669]]
[[0.01046958]]
[[0.00612592]]
[[0.00333974]]
[[0.00363969]]
[[0.00225341]]
[[0.00206323]]
[[0.00084951]]
[[0.00129448]]
[[0.00115637]]
[[0.00149582]]
[[0.02169397]]
[[0.00623027